### Producer 3
This program loads all data from the hotspot_AQUA_streaming.csv, and randomly feed the data to the stream every n seconds, where n is a random number. Additional information such as producer info and created date is attached.


In [1]:
%pip install kafka

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from time import sleep
from json import dumps
import json
from kafka3 import KafkaProducer
import random
import datetime as dt
import pandas as pd
from pymongo import MongoClient

hostip = "10.192.104.132"
client = MongoClient(hostip, 27017)
db = client["fit3182_assignment_db"]

climate_data_collection = db["climate_data"]
topic = 'PartB'

#referenced from week 10 lab, producer 1
def publish_message(producer_instance, topic_name, key, value):
    try:
        

        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(value))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    

if __name__ == '__main__':
    print('Publishing records..')
    producer = connect_kafka_producer()
    secs = 15

    hotspot_data = pd.read_csv('/home/student/A2/Dataset/hotspot_TERRA_streaming.csv').to_dict(orient='records')

    for record in hotspot_data:
        selected_record = random.choice(hotspot_data)
        selected_record['producer'] = 'hotspot_terra'
        
        
        # Generate random timestamp within current day
        random_hour = random.randint(0, 23)
        random_minute = random.randint(0, 59)
        random_second = random.randint(0, 59)
        random_time = dt.datetime.now().replace(hour=random_hour, minute=random_minute, second=random_second)
        
        # Create message with random timestamp
        selected_record['created_time'] = random_time.strftime('%H:%M:%S')

        # Publish the record to Kafka
        publish_message(producer, topic, 'parsed', dumps(selected_record))

        sleep(secs)

Publishing records..
Message published successfully. Data: {"latitude": -38.0233, "longitude": 146.5253, "confidence": 94, "surface_temperature_celcius": 103, "producer": "hotspot_terra", "created_time": "13:26:53"}
Message published successfully. Data: {"latitude": -35.6819, "longitude": 143.5275, "confidence": 91, "surface_temperature_celcius": 69, "producer": "hotspot_terra", "created_time": "23:54:14"}
Message published successfully. Data: {"latitude": -36.477, "longitude": 144.5887, "confidence": 100, "surface_temperature_celcius": 101, "producer": "hotspot_terra", "created_time": "10:54:17"}
Message published successfully. Data: {"latitude": -36.4422, "longitude": 141.427, "confidence": 54, "surface_temperature_celcius": 39, "producer": "hotspot_terra", "created_time": "23:14:14"}
Message published successfully. Data: {"latitude": -36.5104, "longitude": 144.4674, "confidence": 88, "surface_temperature_celcius": 64, "producer": "hotspot_terra", "created_time": "08:55:12"}
Message 